In [ ]:
import requests

NAME = "AT2024ahyy"
TNS_ID = NAME.removeprefix("AT")
tns_data = requests.get("https://tns.snad.space/api/v1/object", params={"name": TNS_ID}).json()
RA, DEC = tns_data["ra"], tns_data["declination"]

In [ ]:
import lsdb

cat = lsdb.open_catalog(
    "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
    search_filter=lsdb.ConeSearch(RA, DEC, radius_arcsec=0.2)
)
df = cat.compute()
assert len(df) == 1, f"{len(df) = }"
row = df.iloc[0]
row

In [ ]:
lc = row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
)
lc

In [ ]:
import numpy as np
from astropy.time import Time
from astropy.units import ABmag, nJy

DECAM_BAND = "i"
DECAM_MAG = 23.171
DECAM_MAG_ERR = 0.238
DECAM_FLUX = (DECAM_MAG * ABmag).to_value(nJy)
DECAM_FLUX_ERR_LOWER = DECAM_FLUX - ((DECAM_MAG + DECAM_MAG_ERR) * ABmag).to_value(nJy)
DECAM_FLUX_ERR_UPPER = ((DECAM_MAG - DECAM_MAG_ERR) * ABmag).to_value(nJy) - DECAM_FLUX
DECAM_JD = 2460647.3159722
DECAM_MJD = Time(DECAM_JD, format="jd").mjd
DECAM_NONDET_JD = 2460641.4841435
DECAM_NONDET_MJD = Time(DECAM_NONDET_JD, format="jd").mjd

NONDET_MJD = np.round(DECAM_NONDET_MJD - 5)
NONDET_MJD

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

for band in "ugrizy":
    data = lc.query("band == @band and psfMagErr < 1")
    plt.errorbar(data["midpointMjdTai"], data["psfMag"], yerr=data["psfMagErr"], label=f"{band}", fmt='o', color=COLORS[band])

plt.errorbar(
    DECAM_MJD, DECAM_MAG, yerr=DECAM_MAG_ERR,
    label=f"DECam {DECAM_BAND}", fmt="s", color=COLORS[DECAM_BAND], markeredgecolor="black", markersize=8,
)

plt.gca().invert_yaxis()
plt.xlabel(f"MJD")
plt.ylabel("PSF mag")
plt.ylim([None, 19.7,])
plt.title(f"{NAME}\ndiaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='upper left')
plt.savefig(f"{row.diaObjectId}_mag.pdf", bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

baseline_flux = {}
baseline_fluxerr = {}
for band in "ugrizy":
    data = lc.query("band == @band and midpointMjdTai < @NONDET_MJD")
    baseline_flux[band] = np.average(data["psfDiffFlux"], weights=data["psfDiffFluxErr"]**-2)
    baseline_fluxerr[band] = np.hypot(
        1.0 / np.sqrt(np.mean(data["psfDiffFluxErr"]**-2)),
        np.std(data["psfDiffFlux"], ddof=1),
    )
    print(f"Baseline {band} flux = {baseline_flux[band]:.0f} ± {baseline_fluxerr[band]:.0f}")

lc["adjusted_diaflux"] = lc["psfDiffFlux"] - np.vectorize(baseline_flux.get)(lc["band"])
lc["s2n"] = lc["adjusted_diaflux"] / np.hypot(lc["psfDiffFluxErr"], np.vectorize(baseline_fluxerr.get)(lc["band"]))

for band in "ugrizy":
    data = lc.query("band == @band")
    label = f"{band}${-baseline_flux[band]:+.0f}$ nJy"
    for alpha, d in [(0.1, data.query("s2n < 3.0")), (1.0, data.query("s2n >= 3.0"))]:
        plt.errorbar(
            d["midpointMjdTai"], d["adjusted_diaflux"], yerr=d["psfDiffFluxErr"],
            label=label if alpha == 1.0 else None, fmt='o', color=COLORS[band], alpha=alpha,
        )
plt.errorbar(
    DECAM_MJD, DECAM_FLUX, yerr=[[DECAM_FLUX_ERR_LOWER], [DECAM_FLUX_ERR_UPPER]],
    label=f"DECam {DECAM_BAND}", fmt="s", color=COLORS[DECAM_BAND], markeredgecolor="black", markersize=8,
)

plt.xlabel(f"MJD")
plt.ylabel("Adjusted PSF DIA Flux, nJy")
plt.ylim([-2500, 5500])
plt.title(f"{NAME}\ndiaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='upper left')
plt.savefig(f"{row.diaObjectId}_flux.pdf", bbox_inches='tight')

In [ ]:
lc.query("s2n >= 3.0").shape

In [ ]:
idx_first_detection = lc.query("s2n >= 3.0")["midpointMjdTai"].idxmin()
print(f'First detection: MJD = {lc.loc[idx_first_detection]["midpointMjdTai"]:.3f}, band = {lc.loc[idx_first_detection]["band"]}, mag = {lc.loc[idx_first_detection]["psfMag"]:.2f} ± {lc.loc[idx_first_detection]["psfMagErr"]:.2f}')